In [1]:
# !pip install -U torchdata torch jiwer torchvision

In [2]:
# !git init .
# !git remote add origin https://github.com/tuanio/image2latex.git
# !git pull origin main

In [3]:
import pandas as pd
import torch
from torch.utils.checkpoint import checkpoint
from torch.utils.data import DataLoader, Dataset
from torch import nn, Tensor
# import torchdata.datapipes as dp
import matplotlib.pyplot as plt
from image2latex import Image2Latex, Text
from pathlib import Path
import torchvision
from torch.nn.utils.rnn import pad_sequence, pad_packed_sequence
import glob
from tqdm.notebook import tqdm
import time
from jiwer import wer as cal_wer
from nltk.metrics import edit_distance
from typing import Tuple
import pytorch_lightning as pl
from tri_stage_lr_scheduler import TriStageLRScheduler

/home/admin/anaconda3/envs/img2latex_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
data_path = Path('im2latex100k')
img_path = Path('im2latex100k/formula_images_processed/formula_images_processed')

In [5]:
bs = 4
# mini_bs = 2
lr = 1e-3
epochs = 15
max_length = 150
log_idx = 300
workers = 12

cuda = torch.cuda.is_available()  
device = torch.device('cuda' if cuda else 'cpu')
# device = 'cpu'
device

device(type='cuda')

In [6]:
text = Text()
n_class = len(text.tokens)
n_class

520

In [7]:
class LatexDataset(Dataset):
    def __init__(self, data_type: str):
        super().__init__()
        assert data_type in ['train', 'test', 'validate'], 'Not found data type'
        csv_path = data_path / f'im2latex_{data_type}.csv'
        df = pd.read_csv(csv_path)
        df['image'] = df.image.map(lambda x: img_path / x)
        self.walker = df.to_dict('records')
        
    def __len__(self):
        return len(self.walker)
    
    def __getitem__(self, idx):
        item = self.walker[idx]
        
        formula = item['formula']
        image = torchvision.io.read_image(str(item['image']))
        
        return image, formula

In [8]:
train_set = LatexDataset('train')
val_set = LatexDataset('validate')
test_set = LatexDataset('test')

len(train_set), len(val_set), len(test_set)

(75275, 8370, 10355)

In [9]:
steps_per_epoch = round(len(train_set) / bs)

warmup_epochs = 2
constant_epochs = 8
decay_epochs = 5

assert warmup_epochs + constant_epochs + decay_epochs == epochs, "Not equal"

steps_per_epoch

18819

In [10]:
def collate_fn(batch):
    formulas = [text.text2int(i[1]) for i in batch]
    formulas = pad_sequence(formulas, batch_first=True)
    sos = torch.zeros(bs, 1) + text.map_tokens['<s>']
    eos = torch.zeros(bs, 1) + text.map_tokens['<e>']
    formulas = torch.cat((sos, formulas, eos), dim=-1).to(dtype=torch.long)
    image = [i[0] for i in batch]
    max_width, max_height = 0, 0
    for img in image:
        c, h, w = img.size()
        max_width = max(max_width, w)
        max_height = max(max_height, h)
    pad = torchvision.transforms.Resize(size=(max_height, max_width))
    image = torch.stack(list(map(lambda x: pad(x), image))).to(dtype=torch.float)
    return image, formulas


class DataModule(pl.LightningDataModule):
    def __init__(self, train_set, val_set, test_set):
        super().__init__()
        self.train_set = train_set
        self.val_set = val_set
        self.test_set = test_set

    def train_dataloader(self):
        return DataLoader(self.train_set,
                          shuffle=True, 
                          batch_size=bs,
                          num_workers=workers,
                          collate_fn=collate_fn)
    
    def val_dataloader(self):
        return DataLoader(self.val_set,
                          shuffle=False, 
                          batch_size=bs,
                          num_workers=workers,
                          collate_fn=collate_fn)
    
    def test_dataloader(self):
        return DataLoader(self.test_set,
                          shuffle=False, 
                          batch_size=bs,
                          num_workers=workers,
                          collate_fn=collate_fn)

In [11]:
class Image2LatexModel(pl.LightningModule):
    def __init__(self, lr=lr, **kwargs):
        super().__init__()
        self.model = Image2Latex(**kwargs)
        self.criterion = nn.CrossEntropyLoss() 
        self.lr = lr
        
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        scheduler = TriStageLRScheduler(optimizer,
                                        init_lr=1e-4,
                                        peak_lr=1e-3,
                                        final_lr=1e-5,
                                        init_lr_scale=0.01,
                                        final_lr_scale=0.01,
                                        warmup_steps=steps_per_epoch * warmup_epochs,
                                        hold_steps=steps_per_epoch * constant_epochs,
                                        decay_steps=steps_per_epoch * decay_epochs,
                                        total_steps=steps_per_epoch * bs)
        scheduler = {
            'scheduler': scheduler,
            'interval': 'step', # or 'epoch'
            'frequency': 1
        }
        return [optimizer], [scheduler]
    
    def forward(self, images, formulas):
        return self.model(images, formulas)

    def training_step(self, batch, batch_idx):
        images, formulas = batch
        
        formulas_in = formulas[:, :-1]
        formulas_out = formulas[:, 1:]
        
        outputs = self.model(images, formulas_in)
        
        bs, t, _ = outputs.size()
        _o = outputs.reshape(bs * t, -1)
        _t = formulas_out.reshape(-1)
        loss = self.criterion(_o, _t)
        
        self.log("train loss", loss)
        
        return loss
    
    def validation_step(self, batch, batch_idx):
        images, formulas = batch
        
        formulas_in = formulas[:, :-1]
        formulas_out = formulas[:, 1:]

        outputs = self.model(images, formulas_in)

        bs, t, _ = outputs.size()
        _o = outputs.reshape(bs * t, -1)
        _t = formulas_out.reshape(-1)
        loss = self.criterion(_o, _t)
        perplexity = torch.exp(loss)

        predicts = [text.tokenize(self.model.decode(i.unsqueeze(0), max_length)) for i in images]
        truths = [text.tokenize(text.int2text(i)) for i in formulas]

        edit_dist = torch.mean(torch.Tensor([edit_distance(pre, tru) / max(len(pre), len(tru)) for pre, tru in zip(predicts, truths)]))
        
        self.log('val loss', loss)
        self.log('val perplexity', perplexity)
        self.log('val edit distance', edit_dist)
    
        return loss
    
    def test_step(self, batch, batch_idx):
        images, formulas = batch
        
        formulas_in = formulas[:, :-1]
        formulas_out = formulas[:, 1:]

        outputs = self.model(images, formulas_in)

        bs, t, _ = outputs.size()
        _o = outputs.reshape(bs * t, -1)
        _t = formulas_out.reshape(-1)
        loss = self.criterion(_o, _t)
        perplexity = torch.exp(loss)

        predicts = [text.tokenize(self.model.decode(i.unsqueeze(0), max_length)) for i in images]
        truths = [text.tokenize(text.int2text(i)) for i in formulas]

        edit_dist = torch.mean(torch.Tensor([edit_distance(pre, tru) / max(len(pre), len(tru)) for pre, tru in zip(predicts, truths)]))
        
        self.log('test loss', loss)
        self.log('test perplexity', perplexity)
        self.log('test edit distance', edit_dist)
    
        return loss

In [12]:
dm = DataModule(train_set, val_set, test_set)

In [13]:
emb_dim = 80
dec_dim = 256
enc_dim = 256
attn_dim = 256

model = Image2LatexModel(lr=lr, n_class=n_class, emb_dim=emb_dim, enc_dim=enc_dim,
                         dec_dim=dec_dim, attn_dim=attn_dim, text=text)

/home/admin/anaconda3/envs/img2latex_env/lib/python3.9/site-packages/torch/nn/modules/rnn.py:67: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [ ]:
tb_logger = pl.loggers.tensorboard.TensorBoardLogger('tb_logs', name='image2latex_model')
lr_monitor = pl.callbacks.LearningRateMonitor(logging_interval='step')

trainer = pl.Trainer(logger=tb_logger, callbacks=[lr_monitor],
                     max_epochs=epochs, accelerator='gpu', accumulate_grad_batches=16)
trainer.fit(datamodule=dm, model=model)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params
-----------------------------------------------
0 | model     | Image2Latex      | 4.5 M 
1 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
4.5 M     Trainable params
0         Non-trainable params
4.5 M     Total params
17.995    Total estimated model params size (MB)


Epoch 0:   2%|██▋                                                                                                                                                     | 363/20912 [01:09<1:05:16,  5.25it/s, loss=6.25, v_num=1]